In [ ]:
sheet_name='SheetName'
worksheet_name = 'WorkSheetName'
key = "KeyName.json"

In [10]:
db = 'DBName'

In [11]:
import pyodbc
import pandas as pd

def connect(database_name):
    '''
    Connection string for ms sql to the database provided
    '''
    con = pyodbc.connect(Driver='{SQL Server}',
                         Server='Name',
                         Database='{}'.format(database_name),
#                          Trusted_Connection=True)
                         User='Name',
                         Password='password')
    return con

            #####################################
def database_table_list(database_name):
    '''
    List's the names of tables in the database
    '''
    sql = """
    SELECT * FROM information_schema.tables
    """
    database_df = pd.read_sql(sql,connect(database_name))
    connect(database_name).close()
    i = 0
    l = []
    while i < len(database_df['TABLE_NAME']):
        l.append(database_df['TABLE_NAME'][i])
        i += 1
    return l

            #####################################
def get_dataframe(table_name,database_name,index_col=None,columns=None):
    '''
    get the dataframe of a table from a database in the database
    '''
    sql = 'select * from {}'
    df = pd.read_sql(sql.format(table_name),connect(database_name),index_col=index_col,columns=columns)
    connect(database_name).close()
    return df

            #####################################
def database_table_columns(database_name):
    '''
    get the table list and columns of those tables in a dictionary in the database
    '''
    i = 0
    dic = {}
    while i < len(database_table_list(database_name)):
        table = database_table_list(database_name)[i]
        i += 1
        a = [get_dataframe(table,database_name).shape, get_dataframe(table,database_name).columns]
        dic[table]=a;
    return dic

            #####################################
def database_number_of_entries(database_name):
    '''
    get the print of all the tables and the entries in them i.e. shape of the table in the database
    '''
    col_name = database_table_columns(database_name)
    table_list = database_table_list(database_name)
    for i in range(len(col_name)):
                   print(table_list[i], ': ', col_name[table_list[i]][0])
                   i += 1
    return 'Total '+str(len(table_list))+' tables'

            #####################################
def database_table_column_text(database_name):
    '''
    Prints the Table name along with the columns of respective tables in the database
    '''
    dataset = database_table_columns(database_name)
    table_list = database_table_list(database_name)
    i=0
    while i < len(database_table_list(database_name)):
        print('\n', database_table_list(database_name)[i], ' : \n\t',end='')
        x=0
        while x < len(dataset[database_table_list(database_name)[i]][1]):
            print(dataset[database_table_list(database_name)[i]][1][x], end="  ")
            x += 1
        i += 1
        
            #####################################


# Getting all records in the dataframe

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# 

In [12]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread

def get_client(key):
    scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
                 "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(key, scope)
    client = gspread.authorize(creds)
    return client

                    ##########################################
def sheets(sheet_name,key,worksheet_name='Sheet1'):
    client = get_client(key)

    try:
        sheet = client.open(sheet_name)
        sheet = sheet.worksheet(worksheet_name)
    except StopIteration:
        sheet = client.open(sheet_name).sheet1
    
    return sheet

                    ##########################################
def get_woksheet_list(sheet_name,key):
    client = get_client(key)
    
    sheet = client.open(sheet_name)
    sheet_list = []
    a = sheet.worksheets()
    for i in range(len(a)):
        sheet_list.append(a[i].title)
    return sheet_list

                    ##########################################
def sheets_to_dataframe(sheet_name,key,file_name='Test',worksheet_name='Sheet1'):
    
    sheet = sheets(sheet_name,key,worksheet_name)
        
    data = sheet.get_all_values()
    headers = data.pop(0)

    df = pd.DataFrame(data, columns=headers)
    writer = pd.ExcelWriter("{}.xlsx".format(file_name), engine='xlsxwriter')
    df.to_excel(writer, sheet_name='Sheet1')
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']
    writer.save()
    df = pd.read_excel('./{}.xlsx'.format(file_name))
    return df


                    ##########################################
def sheets_to_datalist(sheetName, worksheet_name):
    client = get_client(key)
    sheet = client.open(sheetName).worksheet(worksheet_name)
    return sheet.get_all_values()[1:]


                    ##########################################
def sheets_add_column_names(column_list,sheet_name,key,worksheet_name='Sheet1'):

    sheet = sheets(sheet_name,key,worksheet_name)

    sheet.insert_row(column_list,1)
    return 'Done'

                    ##########################################
def sheets_add_record(rows_list,sheet_name,key,worksheet_name='Sheet1'):
    
    sheet = sheets(sheet_name,key,worksheet_name)

    sheet.append_rows(rows_list)
    print('Done')
    return None

                    ##########################################
def sheet_empty(sheet_name,key=key):
    client = get_client(key)
    sheet = client.open(sheet_name)
    c = 0
    for i in sheet.worksheets():
        try:
            sheet.del_worksheet(i)
            print(i.title+" Deleted")
            c+=1
        except:
            print(i.title+'  Left')
            pass
    sheet.add_worksheet('empty',rows=10,cols=10)
    sheet.del_worksheet(i)
    c+=1
    print(str(c-1)+' Sheets deleted')

                    ##########################################
def sheets_import_colums(tables,db,sheet_name,key=key):
    c = 0
    for table in tables:
        try:
            sheet = sheets(sheet_name,key,table)
            sheet.clear()
        except:
            print(table+" Created")
            client = get_client(key)
            sheet = client.open(sheet_name)
            sheet.add_worksheet(table,rows=10,cols=10)
        df = get_dataframe(table,db)
        cols = []
        for i in df.columns:
            cols.append(i)
        sheets_add_column_names(cols,sheet_name,key,table)
        c+=1
    print(str(c)+" table columns imported")
        
        
                    ##########################################
def sheets_import_data(tables,db,sheet_name,key=key):
    '''Fetches the table details from the sql db 
    and imports it to the respective sheets
    tables are the list of tables in sql to import'''
    c = 0
    for table in tables:
        try:
            sheet = sheets(sheet_name,key,table)
            sheet.clear()
        except:
            print(table+" Created")
            client = get_client(key)
            sheet = client.open(sheet_name)
            sheet.add_worksheet(table,rows=10,cols=10)
        df = get_dataframe(table,db)
        cols = []
        for i in df.columns:
            cols.append(i)
        sheets_add_column_names(cols,sheet_name,key,table)
        df = get_dataframe(table,db)
        row = df.values
        rows = []
        for i in range(len(row)):
            a = []
            for j in row[i]:
                a.append(j)
            rows.append(a)
        print(table)
        try:
            sheets_add_record(rows,sheet_name,key,table)
            print('')
        except TypeError:
            print(TypeError)
            print('Empty Table Created \n')
            pass
        except:
            print('api error - jason data type \n') 
            pass
        c += 1
    print('End '+str(c)+' Sheets Created \n')

# 

# 

In [13]:
from random import choice, sample, random
import pandas as pd
from datetime import date
  
def calculateAge(born):
    '''
    Calculate age
    '''
    today = date.today()
    try: 
        birthday = born.replace(year = today.year)
  
    # raised when birth date is February 29
    # and the current year is not a leap year
    except ValueError: 
        birthday = born.replace(year = today.year,
                  month = born.month + 1, day = 1)
  
    if birthday > today:
        return today.year - born.year - 1
    else:
        return today.year - born.year          
# # Driver code
# print(calculateAge(d[1]), "years")

            #####################################

def random_dates(start_year,end_year,value_count=3000):
    '''
    creates a random birthday/date within a given range of years, default count of values 3000
    '''
    dates = []
    y = range(start_year,end_year+1)
    years=[]
    for i in y:
        years.append(i)
    i = 0
    for i in range(0,value_count):
        month = choice(range(1,13))
        day = choice(range(1,32))
        year = choice(years)
        today = date.today()
        if year == today.year:                                             #Current year
            if month > today.month:
                month = choice(range(1,today.month+1))
            if month == today.month and day > today.day:
                day = choice(range(1,today.day+1))
        if (year%4) == 0:                                                 #leapyear
            if month == 2:
                if day in [30,31]:
                    day = 29
                    dates.append(str(month)+'/'+str(day)+'/'+str(year))
            elif month in [4,6,9,11]:
                if day == 31:
                    day=choice(range(1,31))
                    dates.append(str(month)+'/'+str(day)+'/'+str(year))
            else:
                dates.append(str(month)+'/'+str(day)+'/'+str(year))    
        else:
            if month == 2:
                if day in [29,30,31]:                                       #feb
                    day = 28
                    dates.append(str(month)+'/'+str(day)+'/'+str(year))    
            elif month in [4,6,9,11]:
                if day == 31:
                    day=choice(range(1,31))
                    dates.append(str(month)+'/'+str(day)+'/'+str(year))
            else:
                dates.append(str(month)+'/'+str(day)+'/'+str(year))
        i+=1
    return dates


            #####################################
def random_death_dates(start_year,end_year,value_count=2000,rand=8,acceptable_list=[1]):
    '''
    creates a random death date within a given range of years, default count of values 2000
    rand = 8 gives near 300-350,10 near 250,290 near , 50 near 70-80 values
    '''
    dates=[]
    y = range(start_year,end_year+1)
    i=1
    years=[]
    count_no = 0
    randomizer = range(0,rand)
    today = date.today()    
    for i in y:
        years.append(i)
        i+=1
    i = 0
    for i in range(0,value_count):
        month = choice(range(1,13))
        day = choice(range(1,32))
        year = choice(years)
        if year == today.year:                                       #Current Year
            if month > today.month:
                month = choice(range(1,today.month+1))
            if month == today.month and day > today.day:
                day = choice(range(1,today.day+1))
        if choice(randomizer) in acceptable_list:
            count_no+=1
            if (year%4) == 0:                                                 #leapyear
                if month == 2:
                    if day in [30,31]:
                        day = 29
                        dates.append(str(month)+'/'+str(day)+'/'+str(year))
                elif month in [4,6,9,11]:
                    if day == 31:
                        day=choice(range(1,31))
                        dates.append(str(month)+'/'+str(day)+'/'+str(year))
                else:
                    dates.append(str(month)+'/'+str(day)+'/'+str(year))    
            else:
                if month == 2:
                    if day in [29,30,31]:                                       #feb
                        day = 28
                        dates.append(str(month)+'/'+str(day)+'/'+str(year))    
                elif month in [4,6,9,11]:
                    if day == 31:
                        day=choice(range(1,31))
                        dates.append(str(month)+'/'+str(day)+'/'+str(year))
                else:
                    dates.append(str(month)+'/'+str(day)+'/'+str(year))        
        else:
            dates.append('')
        i+=1
    return dates,count_no

            #####################################

def random_time(start,end,count = 3000):

    time = []
    h = range(start,end)
    m = range(0,60)
    s = range(0,60)    
    ms = range(0,1000)
    for i in range(count):
        hr = str(choice(h))
        mins = str(choice(m))
        sec = str(choice(s))
        milisec = str(choice(ms))
        if len(milisec) == 2:
            milisec= '0'+milisec
        if len(milisec) == 1:
            milisec= '00'+milisec
        if len(mins) == 1:
            mins = '0'+mins
        if len(sec) == 1:
            sec = '0'+sec
        time.append(hr+':'+mins+':'+sec)#+'.'+milisec)
    return time

            #####################################

def random_names(file_path,name_count=3000):
    df = pd.read_csv(file_path)
    df = df.dropna()
    a=[]
    i=0
    for i in range(0,name_count):
        x = choice(df.values)[0]
        a.append(x)
#         print(x)
        i+=1
    return a

            #####################################
def random_first_name(full_name_list):
    a = 0
    l = []
    for i in full_name_list:
        i = i.split(' ')
        i = i[0]
        i = i.split('@')[0]
        if i == 'mr.' or i == 'mo.' or i == 'md.' or i == 'mohd.' or i == 'mohd' or i == 'mho' or i == 'mho.' or i == 'sh.' or i == 'dr.' or i == 'ct.' or i == 'ct' or i == 'md' or i == 'sh' or i == 'b' or i == 'mh' or i == 'm' or i == 'na' or i == 'km' or i == 'vi' or i== 'ku.' or i== 'km0' or i== 'smt' or i== 'mrs' or i== 'km.' or i == 'smt.' or i == 'n.' or i == 'dd' or i == 'modh.' or i == 'modh' or i == "p.":
            pass
        else:
            a += 1
            l.append(i)
    print(a)
    l = sample(l,len(l))
    return l

            #####################################
def random_last_name(full_name_list):
    a = 0
    l = []
    for i in full_name_list:
        i = i.split(" ")
        if len(i)<2:
            pass
        elif i[1]=="":
            pass
        elif i[1] == 's/o' or i[1] == 's' or i[1] == 'v' or i[1] == '@' or i[1] == 'u' or i[1] == '@' or i[1] == '&' or i[1] == 'd/o' or i[1] == 'w/o' :
            pass
        elif i[1].endswith('@') or i[1].startswith('@'):
            a+=1
            l.append(i[1].strip('@'))
        else:
            a+=1
            l.append(i[1])
    l = sample(l,len(l))
    print(a)
    return l

            #####################################
def sequence_word(word,length):
    i=1
    a=[]
    for i in range(1,length+1):
        a.append(word+str(i))
    return a

            #####################################

            #####################################
def random_village_region_street_house(tcount=3000):
    h = sample(range(1,27),26)
    s = sequence_word('street',6)
    r = sequence_word('region',7)
    v = sequence_word('village',4)
    var='A'
    alphabets=[]
    alphabets=sample([(chr(ord(var)+i)) for i in range(26)],26)
    village = []
    region = []
    street = []
    house = []
    ''' VILLAGE '''
    i = 0
    while i < 4:
        j=0
        while j < 757:
            village.append(v[i])
            j+=1
        i+=1
    village = sample(village,len(village))
    ''' REGION '''
    i=0
    x=[]
    while i < 7:
        j=0
        while j < 108:
            x.append(r[i])
            j+=1
        i+=1
    region.extend(sample(x,len(x)))
    region.extend(sample(x,len(x)))
    region.extend(sample(x,len(x)))
    region.extend(sample(x,len(x)))
    ''' STREET '''
    i=0
    x=[]
    while i<6:
        j=0
        while j < 18:
            x.append(s[i])
            j+=1
        i+=1
    i=0
    while i<28:
        street.extend(sample(x,len(x)))
        i+=1
    '''HOUSE'''
    j=0
    x=[]
    while j<tcount//28+1:
        i=choice(range(0,25))
        x.append(str(h[i])+alphabets[i])
        j+=1
    i=0
    while i<28:
        house.extend(sample(x,len(x)))
        i+=1

    return (village), (region), (street), (house)

            #####################################
def random_marital(count=3000):
    
    '''
    i=1
    while i <1000:
        if i%14==0:
            print(random_marital(3028).count(''),end='\n')
        else:
            print(random_marital(3028).count(''),end=' ')
        i+=1
    '''
#     m = ('Single ','Single ','Single ','Single ','Single ','Single ','Single ','Single ','Single ','Single ','Single ','Single ','Single ','Single ','Single ','Single ','Single ','Single ','Single ','Single '
#          ,'Married ','Married ','Married ','Married ','Married ','Married ','Married ','Married ','Married ','Married ','Married '
#          ,'Widowed ','Widowed ','Widowed ','Widowed ','Widowed ','Widowed ' 
#          ,'Separated ','Separated '
#          ,'Divorced','Divorced',)
    m = (1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
         ,2,2,2,2,2,2,2,2,2,2,2
         ,3,3,3,3,3,3 
         ,4,4
         ,5,5,)    
    i=0
    x=[]
    for i in range(count):
        x.append(choice(m))
    return x

            #####################################
def random_ratio_marital(count=3000):
    i = 1
    while i<count:
        x=random_marital(100)
        if i%5==0:
            print(x.count('Single '),x.count('Married '),x.count('Widowed '),x.count('Separated '),x.count('Divorced'),end='\n')
        else:
            print(x.count('Single '),x.count('Married '),x.count('Widowed '),x.count('Separated '),x.count('Divorced'),end='    ')
        i+=1
    return "Count = "+str(count)

            #####################################
def random_designations(count=3000):
    
    ''' 
    i=1
    while i <1000:
        if i%14==0:
            print(3028-random_designations(3028).count(''),end='\n')
        else:
            print(3028-random_designations(3028).count(''),end=' ')
        i+=1
    '''
    d = (
        'Data Entry Operator','','','','','','','','','','',
        'Computer Operator','','','','','','','','','',
        'Accountant ','','','','','','','','','','',
        'Computer Operator/Data Entry Operator','','','','','','','','','','',
        'Technical Person','','','','','','','','','','',
        'Technical Assistant','','','','','','','','','','',
        'Overseer','','','','','','','','','',
        'Operator','','','','','','','','','','',
        'Office Clerk','','','','','','','','','','',
        'Accredited Engineer','','','','','','','','','','',
        'Auditor','','','','','','','','','','',
        'Accountant DEO','','','','','','','','','',
        'Electrical Technician','','','','','','','','','','',
        'Lower Division Clerk','','','','','','','','','','',
        'Computer Hardware Engineer','','','','','','','','','','',
        'VAT Resources','','','','','','','','','','',
        'CAR Driver','','','','','','','','','','',
        'Clerk Typist','','','','','','','','','',
        'Social Media Executive','','','','','','','','','','',
        'Attendant','','','','','','','','','','',
        'Data Processing Officer Government Contract job','','','','','','','','','','',
        'Back Office Computer Operator','','','','','','','','','',
        'Engineer','','','','','','','','','','',
        'Executive Accountant','','','','','','','','','','',
        'Site Engineer','','','','','','','','','','',
        'Staff Accountant','','','','','','','','','','',
        'Executive Assistant','','','','','','','','','','',
        'fresher','','','','','','','','','','',
        'Senior Accountant\r','','','','','','','','','','',
        'Data Operator','','','','','','','','','','',
        'Document Controller','','','','','','','','','','',
        'Electrician','','','','','','','','','','',
        'Deputy Project Manager','','','','','','','','','','',
        'Project Assistant','','','','','','','','','','',
        'Legal Advisor','','','','','','','','','','',
        'Junior Engineer Civil','','','','','','','','','','',
        'Clerk','','','','','','','','','','',
        'Assistant','','','','','','','','','','',
        'GIS Engineer','','','','','','','','','','',
        'Office Assistant','','','','','','','','','','',
    )
#     d = (
#         'Data Entry Operator',
#         'Computer Operator',
#         'Accountant ',
#         'Computer Operator/Data Entry Operator',
#         'Technical Person',
#         'Technical Assistant',
#         'Overseer',
#         'Operator',
#         'Office Clerk',
#         'Accredited Engineer',
#         'Auditor',
#         'Accountant DEO',
#         'Electrical Technician',
#         'Lower Division Clerk',
#         'Computer Hardware Engineer',
#         'VAT Resources',
#         'CAR Driver',
#         'Clerk Typist',
#         'Social Media Executive',
#         'Attendant',
#         'Data Processing Officer Government Contract job',
#         'Back Office Computer Operator',
#         'Engineer',
#         'Executive Accountant',
#         'Site Engineer',
#         'Staff Accountant',
#         'Executive Assistant',
#         'fresher',
#         'Senior Accountant\r',
#         'Data Operator',
#         'Document Controller',
#         'Electrician',
#         'Deputy Project Manager',
#         'Project Assistant',
#         'Legal Advisor',
#         'Junior Engineer Civil',
#         'Clerk',
#         'Assistant',
#         'GIS Engineer',
#         'Office Assistant'
#     )
    
    i=0
    x=[]
    c=0
    for i in range(count):
        if choice(d)=="":
            x.append(choice(d))
        else:
            c+=1
            x.append(choice(d))
    return x,c


            #####################################
def random_disability(count=3000):
    
    '''
    i=1
    while i <1000:
        if i%14==0:
            print(random_disability(3028).count(''),end='\n')
        else:
            print(random_disability(3028).count(''),end=' ')
        i+=1
    '''
#     m = ('','','',''
#          ,'Blindness','','',''
#          ,'Cerebral Palsy','','',''
#          ,'Low Vision','','',''
#          ,'Locomotor Disability','','',''
#          ,'Leprosy -Cured','','',''
#          ,'Mental Retardation','','',''
#          ,'Mental Illness','','',''
#          ,'Hearing Impairment','','',''
#         )
    m = (1,1,1,1
         ,2,1,1,1
         ,3,1,1,1
         ,4,1,1,1
         ,5,1,1,1
         ,6,1,1,1
         ,7,1,1,1
         ,8,1,1,1
         ,9,1,1,1
        )
    
    i=0
    x=[]
    for i in range(count):
        x.append(choice(m))
    return x

            #####################################
def random_sex(count=3000):
    
    '''
    i=1
    r = random_sex()
    while i <1000:
        if i%9==0:
            print(r.count('Male'),':',r.count('Female'),end='\n')
        else:
            print(r.count('Male'),':',r.count('Female'),end='----')
        i+=1
    '''
    m = (
        'Male','Male','Male','Male'
        ,'Female','Female','Female'
    )
    #m = ('Male','Female')
    
    i=0
    x=[]
    for i in range(count):
        x.append(choice(m))
    return x

            #####################################
def random_year(start,end,value_count=3000):
    y=range(start,end+1)
    year=[]
    r=[]
    for i in y:
        year.append(i)
    i=0
    while i<value_count:
        r.append(choice(year))
        i+=1
    return r

            #####################################
def random_area(start,end,value_count=3000):
    min = start
    max = end
    i=0
    r=[]
    while i<value_count:
        f = min + (max-min)*random()
        r.append(float("{:.3f}".format(f)))
        i+=1
    return r

            #####################################


# 

In [739]:
from cryptography.fernet import Fernet as fer

name = 'Encryption'
crypto_key = fer.generate_key()
crypto_key = crypto_key.decode()
client = get_client(key)
sheet = client.open(sheet_name)

try:
    wb = sheet.worksheet(name)
    wb.clear()
except:
    sheet.add_worksheet(name,rows=1,cols=1)
    wb = sheet.worksheet(name)
finally:
    wb.append_row([crypto_key])

        #########################################################
def encrypt_data(data,key):
    '''
        Encrypt the data
    '''
    k = fer(key)
    try:
        en = k.encrypt(bytes(data))
    except TypeError:
        en = k.encrypt(data.encode()).decode()
    return en

        ##########################################################
# print(login_pass[1])
# en = encrypt_data(login_pass[1],crypto_key)
# print(en)
# print(f(sheet.worksheet(name).get_all_values()[-1][0].encode()).decrypt(en.encode()).decode())    #en

# 

In [ ]:
male='csvfilepath'
female='csvfilepath'

# 

In [ ]:
name_date_count = 50000

In [ ]:
tcount = 6000

In [ ]:
database_table_list(db)

# 

In [ ]:
list_resident = []

In [ ]:
rgp_id=[]
i=[2,2,3,4,4,4,5]
z=0
while z<tcount:
    z+=1
    rgp_id.append(choice(i))
rgp_id.count(2),rgp_id.count(3),rgp_id.count(4),rgp_id.count(5),len(rgp_id)

In [ ]:
rv_id = []
a = [
    45,45,45,45,45,
    46,46,46,
    47,47,47,47,
    48,48,
    49,49,49,49,49,
    50,50,50,
    51,51,51,
    52,51,52,52,52
]
for i in range(tcount):
    rv_id.append(choice(a))
rv_id.count(45),rv_id.count(46),rv_id.count(47),rv_id.count(48),rv_id.count(49),rv_id.count(50),rv_id.count(51),rv_id.count(51),len(rv_id)

In [ ]:
r_id = []
# for i in range(1,tcount+1):
for i in range(277,6277):
    r_id.append(i)
len(r_id)

In [ ]:
x=random_names(male,name_date_count)
father = random_first_name(x)
x=random_names(female,name_date_count)
mother = random_first_name(x)

In [ ]:
for i in range(len(father)-tcount):
    father.pop()
    i+=1
for i in range(len(mother)-tcount):
    mother.pop()
    i+=1
len(father),len(mother)

In [ ]:
x = random_names(male,name_date_count)
males = random_first_name(x)
y = random_names(male,name_date_count)
males_last = random_last_name(y)
x = random_names(female,name_date_count)
females = random_first_name(x)
y = random_names(female,name_date_count)
females_last = random_last_name(y)

In [ ]:
sex = []
i=0
while i<len(males):
    sex.append('Male')
    i+=1
sex1 = []
i=0
while i<len(females_last):
    sex1.append('Female')
    i+=1

In [ ]:
males = list(zip(males,males_last,sex))
females = list(zip(females,females_last,sex1))

In [ ]:
name=[]
for i in males:
    name.append(i)
for i in females:
    name.append(i)
name = sample(name,len(name))
name = sample(name,len(name))
name = sample(name,len(name))
for i in range(len(name)-tcount):
    name.pop()
    i+=1
len(name)

In [ ]:
rfirst_name=[]
rlast_name=[]
rsex=[]
for x,y,z in name:
    rfirst_name.append(x)
    rlast_name.append(y)
    rsex.append(z)
len(rfirst_name),len(rlast_name),len(rsex)

In [ ]:
rbirth_dates = random_dates(1955,2020,name_date_count)

rbirth_dates = sample(rbirth_dates,len(rbirth_dates))
rbirth_dates = sample(rbirth_dates,len(rbirth_dates))
for i in range(len(rbirth_dates)-tcount):
    rbirth_dates.pop()
    i+=1
len(rbirth_dates)

In [ ]:
age = []

dates = pd.to_datetime(rbirth_dates)
for i in range(len(dates)):
    age.append(calculateAge(dates[i]))
len(age)

In [ ]:
c = 0
for i in age:
    if i <18:
        c+=1
c

In [ ]:
birth_place = []

a = [
    'Mawlynnong','Mawlynnong',
    'Hat Nongjri','Hat Nongjri','Hat Nongjri',
    'Dharnai','Dharnai','Dharnai',
    'Amsara','Amsara',
    'Apegaon','Apegaon','Apegaon','Apegaon',
    'Amarapur','Amarapur',
    'Abdullapur','Abdullapur',
    'Alipura','Alipura','Alipura',
]
i = 0
while i<tcount:
    birth_place.append(choice(a))
    i+=1
birth_place.count('Mawlynnong'),birth_place.count('Hat Nongjri'),birth_place.count('Dharnai'),birth_place.count('Amsara'),birth_place.count('Apegaon'),birth_place.count('Amarapur'),birth_place.count('Abdullapur'),birth_place.count('Alipura'),len(birth_place)

In [ ]:
radhaar = []

x = (0,1,2,3,4,5,6,7,8,9,)
i = 0
while i<tcount:
    radhaar.append(str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x)))
    i+=1
len(radhaar)

In [ ]:
var = 'A'
var1 = 'a'
cap_alphabets=[(chr(ord(var)+i)) for i in range(26)]
small_alphabets=[(chr(ord(var1)+i)) for i in range(26)]
num = []
for i in range(9):
    num.append(str(i))
char = []
for i in '~@#$%^&*)(][;/.,=-\`_+|}{:?><"':
    char.append(i)
password = []
for i in cap_alphabets:
    password.append(i)
for j in small_alphabets:
    password.append(j)
for k in num:
    password.append(k)
for l in char:
    password.append(l)
len(password)

In [ ]:
a = [1,2,3,4,5,6,7,8,9,0]
i = 0
rlogin_pass = []
while i<5000:
    if choice(a)==0:
        rlogin_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
    elif choice(a)==1:
        rlogin_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
    elif choice(a)==2:
        rlogin_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
    elif choice(a)==3:
        rlogin_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
    elif choice(a)==4:
        rlogin_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
    elif choice(a)==5:
        rlogin_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
    elif choice(a)==6:
        rlogin_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
    elif choice(a)==7:
        rlogin_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
    elif choice(a)==8:
        rlogin_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
    elif choice(a)==9:
        rlogin_pass.append(choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password)+choice(password))
    i+=1
len(rlogin_pass)

In [ ]:
for i in range(len(rlogin_pass)-tcount):
    rlogin_pass.pop()
    i+=1
len(rlogin_pass)

# 

In [ ]:
rlogin_pass = []
a = 0
for i in rfirst_name:
    rlogin_pass.append(i+str(r_id[a]))
    a+=1
len(rlogin_pass)

In [786]:
login_pass = rlogin_pass
rlogin_pass = []
for i in login_pass:
    rlogin_pass.append(encrypt_data(i,crypto_key))

In [ ]:
x = random_marital(tcount)
marital = sample(x,len(x))
print(marital.count(1),marital.count(2),marital.count(3),marital.count(4),marital.count(5),len(marital),end='\n')

In [ ]:
m = list(zip(age,marital))
marital = []
for i,j in m:
    if i <18:
        j=1
        marital.append(j)
    else:
        marital.append(j)
print(marital.count(1),marital.count(2),marital.count(3),marital.count(4),marital.count(5),len(marital),end='\n')

In [ ]:
x = random_disability(tcount)
disability = sample(x,len(x))
len(disability)

In [ ]:
num = [0,1,2,3,4,5,6,7,8,9]
rmobile = []
i = 0
c = 0
while i<tcount:
    i+=1
    if choice(num) in [1,3,4]:
        ph = '9'+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))
    elif choice(num) in [2,5,7]:
        ph = '7'+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))
    elif choice(num) in [6,8,9]:
        ph = '8'+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))
    else:
        c += 1
        ph = '0'+'2'+'2'+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))
    rmobile.append(int(ph))
len(rmobile),c

In [ ]:
n = [0,1,2,3,4,5,6,7,8,9]
remail = []
for i in rfirst_name:
    if choice(n) in [1,2,3,4,6,7,8]:
        mail = i+'@gmail.com'
    elif choice(n) in [0,9]:
        mail = i+'@yahoo.com'
    elif choice(n)==5:
        mail = i+'@outlook.com'
    else:
        mail = i+'@redit.com'
    remail.append(mail) 
# list(zip(email,first_name))
len(remail)

In [ ]:
title=[]
fath_id=[]
mom_id=[]
add=[]
sp_id = []
sp_name=[]
mid_name=[]
i = 0
while i<tcount:
    i+=1
    title.append('')
    fath_id.append(0)
    mom_id.append(0)
    add.append('')
    sp_id.append(0)
    sp_name.append('')
    mid_name.append('')
len(title)

# 

In [ ]:
list_staff = []

In [ ]:
sgp_id=[]
i=[2,2,2,2,3,3,4,5,5,5]
z=0
while z<tcount:
    z+=1
    sgp_id.append(choice(i))
sgp_id.count(5),sgp_id.count(2),sgp_id.count(3),sgp_id.count(4),len(sgp_id)

In [ ]:
x = random_names(male,name_date_count)
m = random_first_name(x)
y = random_names(male,name_date_count)
ml = random_last_name(y)
x = random_names(female,name_date_count)
f = random_first_name(x)
y = random_names(female,name_date_count)
fl = random_last_name(y)

In [ ]:
males = m
males_last = ml
females = f
females_last = fl

In [ ]:
sex = []
i=0
while i<len(males):
    sex.append('Male')
    i+=1
sex1 = []
i=0
while i<len(females_last):
    sex1.append('Female')
    i+=1

In [ ]:
males = list(zip(males,males_last,sex))
females = list(zip(females,females_last,sex1))

In [ ]:
name=[]
for i in males:
    name.append(i)
for i in females:
    name.append(i)
name = sample(name,len(name))
name = sample(name,len(name))
name = sample(name,len(name))
for i in range(len(name)-tcount):
    name.pop()
    i+=1
len(name)

In [ ]:
sfirst_name=[]
slast_name=[]
ssex=[]
for x,y,z in name:
    sfirst_name.append(x)
    slast_name.append(y)
    ssex.append(z)
len(sfirst_name),len(slast_name),len(ssex)

In [ ]:
d=[]
for i in range(66,106):
    d.append(i)
i=0
x=[]
c=0
for i in range(tcount):
        c+=1
        x.append(choice(d))

In [ ]:
a=[]
for i in range(66,106):
    print(x.count(i),end='  ')

In [ ]:
designation=x
len(designation)

In [ ]:
s_id=[]
# for i in range(1,tcount+1):
for i in range(50,6050):
    s_id.append(i)
len(s_id)

In [ ]:
join_dates = random_dates(1990,2020,name_date_count)
join_dates = sample(join_dates,len(join_dates))
join_dates = sample(join_dates,len(join_dates))
for i in range(len(join_dates)-tcount):
    join_dates.pop()
    i+=1
len(join_dates)

In [ ]:
end_dates,c = random_death_dates(1995,2021,value_count=name_date_count,rand=12,acceptable_list=[0,1,2,3,4,5])
for i in range(len(end_dates)-tcount):
    end_dates.pop()
    i+=1
len(end_dates),end_dates.count("")

In [ ]:
sbirth_dates = random_dates(1955,2002,name_date_count)
sbirth_dates = sample(sbirth_dates,len(sbirth_dates))
sbirth_dates = sample(sbirth_dates,len(sbirth_dates))
for i in range(len(sbirth_dates)-tcount):
    sbirth_dates.pop()
    i+=1
len(sbirth_dates)

In [ ]:
sis_a=[]
for i in end_dates:
    if i == '':
        sis_a.append(True)
    else:
        sis_a.append(False)

In [ ]:
x = (0,1,2,3,4,5,6,7,8,9,)
i = 0
sadhaar = []
while i<tcount:
    sadhaar.append(str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x))+str(choice(x)))
    i+=1
len(sadhaar)

In [ ]:
slogin_pass = []
a = 0
for i in sfirst_name:
    slogin_pass.append(i+str(s_id[a]))
    a+=1
len(slogin_pass)

In [786]:
login_pass = slogin_pass
slogin_pass = []
for i in login_pass:
    slogin_pass.append(encrypt_data(i,crypto_key))

In [ ]:
num = [0,1,2,3,4,5,6,7,8,9]
smobile = []
i = 0
while i<tcount:
    i+=1
    if choice(num) in [1,3,4]:
        ph = '9'+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))
    elif choice(num) in [2,5,7]:
        ph = '7'+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))
    elif choice(num) in [6,8,9]:
        ph = '8'+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))
    else:
        ph = '0'+'2'+'2'+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))+str(choice(num))
    smobile.append(int(ph))
len(smobile)

In [ ]:
n = [0,1,2,3,4,5,6,7,8,9]
semail = []
for i in rfirst_name:
    if choice(n) in [1,2,3,4,6,7,8]:
        mail = i+'@gmail.com'
    elif choice(n) in [0,9]:
        mail = i+'@yahoo.com'
    elif choice(n)==5:
        mail = i+'@outlook.com'
    else:
        mail = i+'@redit.com'
    semail.append(mail) 
# list(zip(email,first_name))
len(semail)

# 

In [ ]:
list_house = []

In [ ]:
h_id = []
hgp_id = []
hv_id = []
res_id = []
r=0
c = 0
li = list(zip(rgp_id,rv_id,r_id))
li = sample(li,len(li))
for i,j,k in li :
    if age[r]>18:
        hgp_id.append(i)
        hv_id.append(j)
        res_id.append(k)
        c+=1
    r+=1
# for i in range(1,c+1):
for i in range(319,319+len(res_id)):
    h_id.append(i)
len(h_id),len(res_id),len(li)

# House_ID for Residents

In [ ]:
rh_id = []
a = list(zip(res_id,h_id))
a.sort()
r = []
h = []
for i,j in a:
    r.append(i)
    h.append(j)
a = 0
for i in r_id:
    if i in r:
        rh_id.append(int(h[a]))
        a+=1
    else:
        rh_id.append('')

In [ ]:
x,y,z,house = random_village_region_street_house(tcount)
house = sample(house,len(house))
len(house),len(x),len(y),len(z)

In [ ]:
for i in range(len(house)-tcount):
    house.pop()
len(house)

In [ ]:
pr_no = []
i = 0
while i < len(h_id):
    if hgp_id[i]==1:
        pr_no.append('ma'+'-'+house[i])
    if hgp_id[i]==2:
        pr_no.append('dh'+'-'+house[i])
    if hgp_id[i]==3:
        pr_no.append('hb'+'-'+house[i])
    if hgp_id[i]==4:
        pr_no.append('ch'+'-'+house[i])
    i+=1
len(pr_no)

In [ ]:
reg_no = []
a = range(0,10)
var = 'A'
alpha=[(chr(ord(var)+i)) for i in range(26)]
alpha = sample(alpha,26)
i = 0
while i < len(h_id):
    if hgp_id[i]==1:
        reg_no.append('MA'+'-'+choice(alpha)+str(choice(a))+choice(alpha)+str(choice(a))+str(choice(a))+str(choice(a))+str(choice(a))+choice(alpha)+choice(alpha)+str(choice(a))+str(choice(a))+choice(alpha))
    if hgp_id[i]==2:
        reg_no.append('DH'+'-'+choice(alpha)+str(choice(a))+choice(alpha)+str(choice(a))+str(choice(a))+str(choice(a))+str(choice(a))+choice(alpha)+choice(alpha)+str(choice(a))+str(choice(a))+choice(alpha))
    if hgp_id[i]==3:
        reg_no.append('HB'+'-'+choice(alpha)+str(choice(a))+choice(alpha)+str(choice(a))+str(choice(a))+str(choice(a))+str(choice(a))+choice(alpha)+choice(alpha)+str(choice(a))+str(choice(a))+choice(alpha))
    if hgp_id[i]==4:
        reg_no.append('CH'+'-'+choice(alpha)+str(choice(a))+choice(alpha)+str(choice(a))+str(choice(a))+str(choice(a))+str(choice(a))+choice(alpha)+choice(alpha)+str(choice(a))+str(choice(a))+choice(alpha))
    i+=1
len(reg_no)

In [ ]:
reg_y = random_year(1975,2020,len(h_id))
len(reg_y)

In [ ]:
a = [.3,.25,.2,.15,.1]
c = []
for i in a:
    c.append(round(len(h_id)*i))
x = 0
for i in c:
    x+=i
c

In [ ]:
ss = random_area(50,500,c[0])
s = random_area(500,1000,c[1])
m = random_area(1000,1500,c[2])
l = random_area(1500,2000,c[3])
xl = random_area(2000,2500,c[4])

In [ ]:
len(ss)+len(s)+len(m)+len(l)+len(xl)

In [ ]:
area = []
for i in ss:
    area.append(i)
for i in s:
    area.append(i)
for i in m:
    area.append(i)
for i in l:
    area.append(i)
for i in xl:
    area.append(i)
len(area)

In [ ]:
area = sample(area,len(area))
area = sample(area,len(area))
area = sample(area,len(area))

In [ ]:
home_tax = []
for i in area:
    if i <500:
        home_tax.append(100)
    elif i<1000:
        home_tax.append(200)
    elif i<1500:
        home_tax.append(400)
    elif i<2000:
        home_tax.append(800)
    else:
        home_tax.append(1000)
len(home_tax)

In [ ]:
water_tax = []
for i in area:
    if i <500:
        water_tax.append(choice(range(10,50)))
    elif i<1000:
        water_tax.append(choice(range(50,100)))
    elif i<1500:
        water_tax.append(choice(range(100,250)))
    elif i<2000:
        water_tax.append(choice(range(250,350)))
    else:
        water_tax.append(choice(range(350,550)))
len(water_tax)

In [ ]:
b_tax = []
for i in area:
    if i <500:
        b_tax.append(choice(range(100,250)))
    elif i<1000:
        b_tax.append(choice(range(250,500)))
    elif i<1500:
        b_tax.append(choice(range(500,750)))
    elif i<2000:
        b_tax.append(choice(range(750,1200)))
    else:
        b_tax.append(choice(range(1200,1800)))
len(b_tax)

In [ ]:
s_tax = []
for i in area:
    if i <500:
        s_tax.append(choice(range(500,1000)))
    elif i<1000:
        s_tax.append(choice(range(1000,1500)))
    elif i<1500:
        s_tax.append(choice(range(1500,2000)))
    elif i<2000:
        s_tax.append(choice(range(2000,2500)))
    else:
        s_tax.append(choice(range(2500,3000)))
len(s_tax)

In [ ]:
light_tax = []
for i in area:
    if i <500:
        light_tax.append(200)
    elif i<1000:
        light_tax.append(500)
    elif i<1500:
        light_tax.append(1500)
    elif i<2000:
        light_tax.append(2000)
    else:
        light_tax.append(2500)
len(light_tax)

In [ ]:
health_tax = []
for i in area:
    if i <500:
        health_tax.append(200)
    elif i<1000:
        health_tax.append(500)
    elif i<1500:
        health_tax.append(1500)
    elif i<2000:
        health_tax.append(2000)
    else:
        health_tax.append(2500)
len(health_tax)

In [ ]:
s_tax = []
for i in area:
    if i <500:
        s_tax.append(200)
    elif i<1000:
        s_tax.append(600)
    elif i<1500:
        s_tax.append(1500)
    elif i<2000:
        s_tax.append(2000)
    else:
        s_tax.append(3000)
len(s_tax)

In [ ]:
a = range(7)
for i in range(10):
    n_fee = []
    c = 0
    for i in area:
        if choice(a) in [0,1,2]:
            '''
            this configuration gives 40% populated'''
            c+=1
            if i <500:
                n_fee.append(10)
            elif i<1000:
                n_fee.append(30)
            elif i<1500:
                n_fee.append(70)
            elif i<2000:
                n_fee.append(110)
            else:
                n_fee.append(200)
        else:
            n_fee.append(0)
    print(c,end=' ')
len(n_fee)

In [ ]:
a = range(5)
for i in range(10):
    w_fee = []
    c = 0
    for i in area:
        if choice(a) in [0]:
            '''
            this configuration gives 20% populated'''
            c+=1
            if i <500:
                w_fee.append(20)
            elif i<1000:
                w_fee.append(60)
            elif i<1500:
                w_fee.append(100)
            elif i<2000:
                w_fee.append(170)
            else:
                w_fee.append(300)
        else:
            w_fee.append(0)
    print(c,end=' ')
len(w_fee)

In [ ]:
a = range(12)
for i in range(10):
    p_amount = []
    c = 0
    for i in area:
        if choice(a) in [0,6,5,2,4,1,3,7]:
            c+=1
            if i <500:
                p_amount.append(choice(range(200)))
            elif i<1000:
                p_amount.append(choice(range(500)))
            elif i<1500:
                p_amount.append(choice(range(1000)))
            elif i<2000:
                p_amount.append(choice(range(1600)))
            else:
                p_amount.append(choice(range(2200)))
        else:
            p_amount.append(0)
    print(c,end=' ')
len(p_amount)

In [ ]:
o_tax = []
for i in area:
    if i <500:
        o_tax.append(choice(range(50)))
    elif i<1000:
        o_tax.append(choice(range(150)))
    elif i<1500:
        o_tax.append(choice(range(300)))
    elif i<2000:
        o_tax.append(choice(range(600)))
    else:
        o_tax.append(choice(range(1200)))
len(o_tax)

In [ ]:
# for i in o_tax:
#     print(o_tax,end=' ')

# 

In [ ]:
list_tax_master = []

In [ ]:
gy = []
gm = []
y = [2020,2021]
m = range(1,13)
for i in range(len(h_id)):
    a = choice(y)
    s = choice(m)
    gy.append(a)
    if a == 2021 and s>5:
        gm.append(choice((1,2,3,4,5)))
    else:
        gm.append(s)

In [ ]:
fhome_tax = home_tax
flight_tax = light_tax
fhealth_tax = health_tax
fwater_tax = water_tax
fb_tax = b_tax
fs_tax = s_tax

In [ ]:
insert_date = random_dates(2019,2021,len(h_id)+3000+10000)
time = random_time(9,18,len(h_id)+3000+10000)
a = list(zip(insert_date,time))
tinsert_date_time = []
for i,j in a:
    tinsert_date_time.append(i+' '+j)
# for i in range(len(a)-len(h_id)):
#     tinsert_date_time.pop()
len(tinsert_date_time)

In [ ]:
for i in tinsert_date_time:
    print(i)

In [ ]:
t_id = []
for i in range(99,99+len(h_id)):
    t_id.append(i)
len(t_id)

In [ ]:
total_other_tax = []
li = list(zip(flight_tax,fhealth_tax,fb_tax,fs_tax,n_fee,w_fee,o_tax))
for i,j,k,l,m,n,o in li:
    total_other_tax.append(i+j+k+l+m+n+o)
len(total_other_tax)

# 

In [ ]:
list_payment =[]

In [ ]:
p_id = []
for i in range(2,2+tcount*2):
    p_id.append(i)

In [ ]:
len(p_id)

In [ ]:
dbm = [2,2,2,2,3,3,3,4,4,]
dbm_id = []
for i in range(len(p_id)):
    dbm_id.append(choice(dbm))
dbm_id.count(0),dbm_id.count(1),dbm_id.count(2),dbm_id.count(3),dbm_id.count(4),len(dbm_id)

In [ ]:
p_id = sample(p_id,len(p_id))

In [ ]:
h_id1 = []
hgp_id1 = []
fhome_tax1 = []
fwater_tax1 = []
total_other_tax1 = []
a = list(zip(h_id,hgp_id,fhome_tax,fwater_tax,total_other_tax))
j=1
i=0
while i<j:
    for x,y,z,a,b in a:
        h_id1.append(x)
        hgp_id1.append(y)
        fhome_tax1.append(z)
        fwater_tax1.append(a)
        total_other_tax1.append(b)
    i+=1

In [ ]:
a

In [ ]:
pgp_id = []
ph_id = []
wtp_id = []
htp_id = []
otp_id = []
p_amount = []
a = list(zip(p_id,h_id1,hgp_id1,fhome_tax1,fwater_tax1,total_other_tax1,dbm_id))
for b,c,d,x,y,z,t in a:
    if t == 1:
        p_amount.append(x+y+z)
        htp_id.append(b)
        wtp_id.append(b)        
        otp_id.append(b)
    elif t == 2:
        p_amount.append(x)
        htp_id.append(b)
        wtp_id.append(0)        
        otp_id.append(0)
    elif t == 3:
        p_amount.append(y)
        htp_id.append(0)
        wtp_id.append(b)        
        otp_id.append(0)
    elif t == 4:
        p_amount.append(z)
        htp_id.append(0)
        wtp_id.append(0)        
        otp_id.append(b)
    pgp_id.append(d)
    ph_id.append(c)
# t_amount = p_amount
a = list(zip(p_id,ph_id,pgp_id,p_amount,dbm_id))

In [ ]:
a.sort()
p_id = []
ph_id = []
pgp_id = []
p_amount = []
dbm_id = []
for b,c,d,x,y in a:
    p_id.append(b)
    ph_id.append(c)    
    pgp_id.append(d)
    p_amount.append(x)    
    dbm_id.append(y)

In [ ]:
update_time = []
d = random_dates(2019,2021,tcount+tcount)
t = random_time(9,18,tcount)
for i in range(len(pgp_id)):
    update_time.append(d[i]+' '+t[i])
update_time = sample(update_time,len(update_time))
len(update_time)#,update_time

In [ ]:
len(ph_id)

In [ ]:
a = range(30)
pis_a = []
for i in range(len(pgp_id)):
    if choice(a) in [0]:
        pis_a.append(False)
    else:
        pis_a.append(True)
pis_a.count(True),pis_a.count(False),len(pis_a)

In [ ]:
a = [1,1,2,3,4,5,6,]
payment_mode = []
for i in range(len(pgp_id)):
    payment_mode.append(choice(a))
len(payment_mode)

In [ ]:
p_date = random_dates(2020,2021,len(pgp_id)+len(pgp_id))
for i in range(len(p_date)-len(pgp_id)):
    p_date.pop()
len(p_date)

In [ ]:
tranpay_id = p_id
tranorder_id = p_id

# 

In [ ]:
a = range(10)
is_a = []
for i in range(6000):
    if choice(a) in [0]:
        is_a.append(False)
    else:
        is_a.append(True)
is_a.count(True),is_a.count(False),len(is_a)

In [ ]:
ris_a = sample(is_a,len(is_a))
list_resident = list(zip(r_id,rgp_id,title,rfirst_name,rlast_name,fath_id,
                         mom_id,rsex,rbirth_dates,birth_place,rh_id,radhaar,
                         add,rlogin_pass,ris_a,marital,disability,sp_id,
                         sp_name,father,mother,mid_name,rmobile,remail,
                         rv_id))

In [ ]:
his_a = sample(is_a,len(is_a))
a = add
for i in range(len(add)-len(h_id)):
    is_a.pop()
    a.pop()
list_house = list(zip(h_id,hgp_id,res_id,pr_no,reg_no,reg_y,area,
                      home_tax,light_tax,health_tax,water_tax,b_tax,
                      s_tax,n_fee,w_fee,o_tax,a,his_a,hv_id))

In [ ]:
tis_a = sample(is_a,len(is_a))
list_tax_master = list(zip(t_id,hgp_id,res_id,h_id,gy,gm,fhome_tax,
                          flight_tax,fhealth_tax,fwater_tax,fb_tax,fs_tax,
                          n_fee,w_fee,o_tax,add,tis_a,tinsert_date_time,t_amount,
                           total_other_tax,htp_id,wtp_id,otp_id))

In [ ]:
list_staff = list(zip(s_id,designation,join_dates,end_dates,sfirst_name,
                      slast_name,ssex,sbirth_dates,sadhaar,add,sis_a,
                      slogin_pass,semail,smobile,sgp_id))

In [ ]:
list_payment = list(zip(p_id,p_date,ph_id,p_amount,payment_mode,pis_a,update_time,tranpay_id,tranorder_id,add,pgp_id,dbm_id))

In [ ]:
c = 1
print('resident')
for i in (r_id,rgp_id,title,rfirst_name,rlast_name,fath_id,
                         mom_id,rsex,rbirth_dates,birth_place,rh_id,radhaar,
                         add,rlogin_pass,ris_a,marital,disability,sp_id,
                         sp_name,father,mother,mid_name,rmobile,remail,
                         rv_id):
    print(str(c) +'-'+str(len(i)),end=' ')
    c+=1

In [ ]:
print('house')
c = 1
for i in (h_id,hgp_id,res_id,pr_no,reg_no,reg_y,area,
                      home_tax,light_tax,health_tax,water_tax,b_tax,
                      s_tax,n_fee,w_fee,o_tax,a,his_a,hv_id):
    print(str(c) +'-'+str(len(i)),end=' ')
    c+=1

In [ ]:
print('tax master')
c = 1
for i in (t_id,hgp_id,res_id,h_id,gy,gm,fhome_tax,
                          flight_tax,fhealth_tax,fwater_tax,fb_tax,fs_tax,
                          n_fee,w_fee,o_tax,add,tis_a,tinsert_date_time,a,
                           total_other_tax,htp_id,wtp_id,otp_id):
    print(str(c) +'-'+str(len(i)),end=' ')
    c+=1

In [ ]:
print('staff')
c = 1
for i in (s_id,designation,join_dates,end_dates,sfirst_name,
                      slast_name,ssex,sbirth_dates,sadhaar,add,sis_a,
                      slogin_pass,semail,smobile,sgp_id):
    print(str(c) +'-'+str(len(i)),end=' ')
    c+=1

In [ ]:
print('payment')
c = 1
for i in (p_id,p_date,ph_id,p_amount,payment_mode,pis_a,update_time,tranpay_id,tranorder_id,add,pgp_id,dbm_id):
    print(str(c) +'-'+str(len(i)),end=' ')
    c+=1

# 

#  WorkSpace

In [ ]:
table = ['Resident','House','Staff','TaxMaster','Payment']
lists = [list_resident,list_house,list_staff,list_tax_master,list_payment]
client = get_client(key)
sheet = client.open(sheet_name)

for i in range(len(table)):
    print('\n'+table[i])
    worksheet = sheet.worksheet(table[i])
    worksheet.clear()
    print("cols")
    sheets_add_column_names(list(database_table_columns(db)[table[i]][1]),sheet_name,key,table[i])
    print("Records")
    sheets_add_record(lists[i],sheet_name,key,table[i])

In [ ]:
# import pyodbc as odbc

# con = odbc.connect(
#         'Driver={SQL Server};'
#         'Server=DEVILSHORN999\DEVILSHORN999;'
#         'Database=Grampanchayat;'
#         'Trusted_Connection=True;'
#     )
con = connect(db)
cursor = con.cursor()
con.autocommit = False

In [ ]:
    names = ['TaxMaster','Payment','Staff']
    name = names[0]
# for name in names:
    print(name)
#     df = sheets_to_dataframe(sheet_name,key,worksheet_name=name)
#     print('Fetched Sheets DataFrame')
#     cols = []
#     c = 0
#     b=[]
#     for i in df.columns:
#         cols.append(i)
#     cols.pop(0)
#     cols.pop(0)
#     df = df[cols]
    print('Column list ready')
#     a = sheets_to_datalist(sheet_name,name)
#     print('Fetched Datalist')
#     for i in range(len(a)):
#         a[i].pop(0)
#         a[i].pop(0)
#         a[i].pop(-4)
    c = ""
    v = ""
    x = 1
    for i in cols:
        if x == len(cols):
            c = c+i
            v = v+'?'
        else:
            c = c+i+','
            v = v+'?,'
        x+=1
    sql = "insert into {} ({}) values({})"
    print(sql.format(name,c,v),'\nExecuting statement')
    cursor.executemany(sql.format(name,c,v),a)
    cursor.commit()
    print(name,' : Done\n')

In [ ]:
con = connect(db)
cursor = con.cursor()
con.autocommit = False

In [ ]:
import datetime
datetime.datetime.strptime(a[0][0], '%m/%d/%Y').isoformat()

In [ ]:
con.close()

In [ ]:
cursor.commit()

# 